In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.6 MB/s eta 0:00:00


In [2]:
!pip install transformers

In [4]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from transformers import T5Tokenizer, T5Model
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features with T5 embeddings
def process_features_t5(input_file, text_column, numeric_features, model_name="t5-small"):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Load T5 tokenizer and model
    print("Loading T5 model...")
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5Model.from_pretrained(model_name)

    # Generate T5 embeddings for each text input
    print("Generating T5 embeddings...")
    embeddings = []
    for text in df[text_column].astype(str).tolist():
        inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
        with torch.no_grad():
            outputs = model.encoder(inputs['input_ids'])
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Mean pooling
            embeddings.append(embedding)
    embeddings = np.array(embeddings)
    print(f"T5 embeddings shape: {embeddings.shape}")

    # Extract numeric features and combine with embeddings
    numeric_df = df[numeric_features]
    combined_features = np.hstack([embeddings, numeric_df.to_numpy()])

    return pd.DataFrame(combined_features), df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Define model functions
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'solver': ['liblinear', 'saga'], 'max_iter': [100, 200]}
    model = tune_model(LogisticRegression(), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'SVM': svm_model,
        'Logistic Regression': logistic_regression_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Process the features with T5
X, original_data = process_features_t5(input_file, text_column, numeric_features)

# Prepare target variable with shifted labels
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models with hyperparameter tuning
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading T5 model...
Generating T5 embeddings...
T5 embeddings shape: (681, 512)
Running KNN model...
Best Params: {'weights': 'distance', 'n_neighbors': 10}
KNN Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7024
Test Precision: 0.6248
Test Recall: 0.7024
Test F1 Score: 0.6564
Test Confusion Matrix:
[[  0   1  12]
 [  0   9  32]
 [  0  16 135]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.6927
Test Precision: 0.6160
Test Recall: 0.6927
Test F1 Score:

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=30. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'gamma': 'scale', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.6576
Train Precision: 0.5714
Train Recall: 0.6576
Train F1 Score: 0.5352
Train Confusion Matrix:
[[  0   1  31]
 [  0   4 128]
 [  0   3 309]]

Test Accuracy: 0.7366
Test Precision: 0.6460
Test Recall: 0.7366
Test F1 Score: 0.6414
Test Confusion Matrix:
[[  0   0  13]
 [  0   2  39]
 [  0   2 149]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=30. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'max_iter': 100, 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.6933
Train Precision: 0.6429
Train Recall: 0.6933
Train F1 Score: 0.6106
Train Confusion Matrix:
[[  0   6  26]
 [  0  23 109]
 [  0   5 307]]

Test Accuracy: 0.7171
Test Precision: 0.6013
Test Recall: 0.7171
Test F1 Score: 0.6380
Test Confusion Matrix:
[[  0   1  12]
 [  0   3  38]
 [  0   7 144]]

Running Decision Tree model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 20}
Decision Tree Model Metrics:
Train Accuracy: 0.9055
Train Precision: 0.9057
Train Recall: 0.9055
Train F1 Score: 0.9051
Train Confusion Matrix:
[[ 20   8   4]
 [  4 118  10]
 [  4  15 293]]

Test Accuracy: 0.5854
Test Precision: 0.6152
Test Recall: 0.5854
Test F1 Score: 0.5983
Test Confusion Matrix:
[[  0   5   8]
 [  4  13  24]
 [  8  36 107]]

Running Naive Bayes model...
Naive Bayes Model Metrics:
Train Accuracy: 0.4244
Train Precision: 0.5834
Train Recall: 0.4244
Train F1 Score: 0.4740
Train Confusion Matrix:
[[ 18   5   9]
 [ 51  36  45]
 [101  63 148]]

Test Accuracy: 0.4341
Test Precision: 0.6346
Test Recall: 0.4341
Test F1 Score: 0.5065
Test Confusion Matrix:
[[ 2  5  6]
 [17 11 13]
 [40 35 76]]

